#Orange Team 
##Data Mining Project

NLP with disaster tweets. Real or not?


In [1]:
# installing the necessary packages
import  numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
import spacy

%matplotlib inline

In [2]:
#read the training data
data = pd.read_csv('https://raw.githubusercontent.com/michalis0/DataMining_and_MachineLearning/master/project/training_data.csv')
data.head()

,id,keyword,location,text,target
0,3738,destroyed,USA,Black Eye 9: A space battle occurred at Star O...,0
1,853,bioterror,NaN,#world FedEx no longer to transport bioterror ...,0
2,10540,windstorm,"Palm Beach County, FL",Reality Training: Train falls off elevated tra...,1
3,5988,hazardous,USA,#Taiwan Grace: expect that large rocks trees m...,1
4,6328,hostage,Australia,New ISIS Video: ISIS Threatens to Behead Croat...,1


In [27]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6471 entries, 0 to 6470
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        6471 non-null   int64 
 1   keyword   6416 non-null   object
 2   location  4330 non-null   object
 3   text      6471 non-null   object
 4   target    6471 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 252.9+ KB


In [29]:
# read the test data and save for later
t_df = pd.read_csv('https://raw.githubusercontent.com/michalis0/DataMining_and_MachineLearning/master/project/test_data.csv')
t_df.head()

,id,keyword,location,text
0,9972,tsunami,NaN,Crptotech tsunami and banks.\n http://t.co/KHz...
1,9865,traumatised,"Portsmouth, UK",I'm that traumatised that I can't even spell p...
2,1937,burning%20buildings,NaN,@foxnewsvideo @AIIAmericanGirI @ANHQDC So ... ...
3,3560,desolate,NaN,Me watching Law &amp; Order (IB: @sauldale305)...
4,2731,crushed,bahstun/porta reeko,Papi absolutely crushed that ball


In [30]:
t_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1142 entries, 0 to 1141
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        1142 non-null   int64 
 1   keyword   1136 non-null   object
 2   location  750 non-null    object
 3   text      1142 non-null   object
dtypes: int64(1), object(3)
memory usage: 35.8+ KB


In [4]:
data.describe()

,id,target
count,6471.000000,6471.000000
mean,5446.289600,0.428064
std,3139.343612,0.494836
min,1.000000,0.000000
25%,2731.000000,0.000000
50%,5450.000000,0.000000
75%,8161.500000,1.000000
max,10873.000000,1.000000


We can see there are 6471 samles to train our model. The target variable is binary.

In [5]:
data.target.value_counts()/data.shape[0]

0    0.571936
1    0.428064
Name: target, dtype: float64

The base rate for the training dataset is 57%. This is the proportion of tweets that are not about disasters.

In [6]:
data.keyword.value_counts()

deluge                   39
earthquake               38
harm                     37
collision                37
hostages                 36
                         ..
demolished               18
threat                   10
epicentre                10
radiation%20emergency     9
inundation                6
Name: keyword, Length: 221, dtype: int64

There are 221 different key words given.

In [7]:
data.location.value_counts()

USA                       91
New York                  59
United States             46
London                    39
Canada                    25
                          ..
London/Outlaw Country      1
316                        1
Bangkok                    1
UK Great Britain           1
taken by piper curda       1
Name: location, Length: 2921, dtype: int64

There are 2921 different locations available.

In [8]:
# Download the english language model
!python -m spacy download en

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


preprocess the tweets:

- Remove the stopwords. Use the stop words from spacy package.

- Remove the punctuation marks. Use the punctuation marks from the string package.

- Lowercase all of the words.

- Lemmatize all of the words. Lemmatize the words using the spacy package, similar to what you did in the lab!


In [9]:
# Load English language model
sp = spacy.load('en_core_web_sm')

In [10]:
# Create a list of stopwords
stop_words = spacy.lang.en.stop_words.STOP_WORDS

list(stop_words)[:10]

['beside',
 'many',
 'so',
 'among',
 'whereafter',
 'once',
 'none',
 'seeming',
 'may',
 '‘ll']

In [11]:
import string
# Create a list of punctuation marks
punctuations = string.punctuation

punctuations

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [12]:
# Create tokenizer function
def spacy_tokenizer(sentence):
    # Create token object, which is used to create documents with linguistic annotations.
    mytokens = sp(sentence)

    # Lemmatize each token and convert each token into lowercase
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]

    # Remove stop words and punctuation
    mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations ]

    # Return preprocessed list of tokens
    return mytokens

In [13]:
# Select features
X = data.text
y = data.target
X.shape

(6471,)

In [14]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=72)
X_train.shape

(5176,)

In [15]:
tfidf_vector = TfidfVectorizer(tokenizer=spacy_tokenizer) # we use the above defined tokenizer
classifier = LogisticRegressionCV(solver="lbfgs", cv=5, max_iter=2000, random_state=72)
pipe = Pipeline([('vectorizer', tfidf_vector), ('classifier', classifier)])

In [16]:
pipe.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('vectorizer',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_patt...
                                 tokenizer=<function spacy_tokenizer at 0x7f45a0d62e18>,
                                 use_idf=True, vocabulary=None)),
                ('classifier',
                 LogisticRegressionCV(Cs=10, class_weight=None, cv=5,
                                      dual=Fal

In [17]:
y_pred = pipe.predict(X_test)

In [18]:
print(f"ACCURACY SCORE:\n{accuracy_score(y_test, y_pred):.4f}")

ACCURACY SCORE:
0.8000


Predictions on the real test dataset (the separate dataset loaded at t_df)

In [24]:
t_df.text.describe()

count                                                  1142
unique                                                 1140
top       CLEARED:incident with injury:I-495  inner loop...
freq                                                      2
Name: text, dtype: object

In [19]:
y_predpred = pipe.predict(t_df.text)

In [20]:
y_predpred =pd.Series(y_predpred)

In [23]:
y_predpred.describe()

count    1142.000000
mean        0.395797
std         0.489235
min         0.000000
25%         0.000000
50%         0.000000
75%         1.000000
max         1.000000
dtype: float64

In [35]:
from google.colab import drive
drive.mount('/drive')

Mounted at /drive


In [37]:
y_predpred.rename('target').to_csv('/drive/My Drive/Colab Notebooks/1st_submission.csv', index = False)